In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Aggregate")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/09/09 20:39:35 INFO SparkContext: Running Spark version 3.5.0
24/09/09 20:39:35 INFO SparkContext: OS info Linux, 6.8.0-40-generic, amd64
24/09/09 20:39:35 INFO SparkContext: Java version 11.0.24
24/09/09 20:39:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/09 20:39:36 INFO ResourceUtils: ==============================================================
24/09/09 20:39:36 INFO ResourceUtils: No custom resources configured for spark.driver.
24/09/09 20:39:36 INFO ResourceUtils: ==============================================================
24/09/09 20:39:36 INFO SparkContext: Submitted application: Aggregate
24/09/09 20:39:36 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap ->

spark: SparkSession = org.apache.spark.sql.SparkSession@590e0b06
import spark.implicits._

In [4]:
val data = Seq(
    ("Jean-Georges", "Perrin", "NC", 1, 300, 1551903533),
    ("Jean-Georges", "Perrin", "NC", 2, 120, 1551903567),
    ("Jean-Georges", "Perrin", "CA" ,4, 75, 1551903599),
    ("Holden", "Karau", "CA" , 6, 37, 1551904299),
    ("Ginni", "Rometty", "NY", 7, 91, 1551916792),
    ("Holden", "Karau", "CA", 4, 153, 1552876129)
).toDF("firstName", "lastName", "state", "quantity", "revenue", "timestamp")

data.show()

24/09/09 20:39:39 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/09/09 20:39:39 INFO SharedState: Warehouse path is 'file:/home/vadim/workspace/Spark/UDF%20UDAF/spark-warehouse'.
24/09/09 20:39:40 INFO CodeGenerator: Code generated in 294.502561 ms
24/09/09 20:39:42 INFO CodeGenerator: Code generated in 14.621223 ms
24/09/09 20:39:42 INFO CodeGenerator: Code generated in 17.643521 ms


+------------+--------+-----+--------+-------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|
+------------+--------+-----+--------+-------+----------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|
|      Holden|   Karau|   CA|       6|     37|1551904299|
|       Ginni| Rometty|   NY|       7|     91|1551916792|
|      Holden|   Karau|   CA|       4|    153|1552876129|
+------------+--------+-----+--------+-------+----------+



data: DataFrame = [firstName: string, lastName: string ... 4 more fields]

In [5]:
val aggDf = data
            .groupBy($"firstName", $"lastName", $"state")
            .agg(
                sum("quantity").as("sum_qty"),
                sum("revenue").as("sum_rev"),
                avg("revenue").as("avg_rev"))

aggDf.show()

24/09/09 20:39:43 INFO CodeGenerator: Code generated in 114.926076 ms
24/09/09 20:39:43 INFO CodeGenerator: Code generated in 20.06419 ms
24/09/09 20:39:43 INFO DAGScheduler: Registering RDD 2 (show at cell5.sc:8) as input to shuffle 0
24/09/09 20:39:43 INFO DAGScheduler: Got map stage job 0 (show at cell5.sc:8) with 6 output partitions
24/09/09 20:39:43 INFO DAGScheduler: Final stage: ShuffleMapStage 0 (show at cell5.sc:8)
24/09/09 20:39:43 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:39:43 INFO DAGScheduler: Missing parents: List()
24/09/09 20:39:43 INFO DAGScheduler: Submitting ShuffleMapStage 0 (MapPartitionsRDD[2] at show at cell5.sc:8), which has no missing parents
24/09/09 20:39:43 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 43.6 KiB, free 4.5 GiB)
24/09/09 20:39:44 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 18.6 KiB, free 4.5 GiB)
24/09/09 20:39:44 INFO BlockManagerInfo: Added broadcas

+------------+--------+-----+-------+-------+-------+
|   firstName|lastName|state|sum_qty|sum_rev|avg_rev|
+------------+--------+-----+-------+-------+-------+
|Jean-Georges|  Perrin|   NC|      3|    420|  210.0|
|Jean-Georges|  Perrin|   CA|      4|     75|   75.0|
|      Holden|   Karau|   CA|     10|    190|   95.0|
|       Ginni| Rometty|   NY|      7|     91|   91.0|
+------------+--------+-----+-------+-------+-------+



aggDf: DataFrame = [firstName: string, lastName: string ... 4 more fields]